Connexion au Drive et Installation

In [4]:
from google.colab import drive
import os
#drive.mount('/content/drive')

!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.92
    Uninstalling opencv-python-headless-4.13.0.92:
      Successfully uninstalled opencv-python-headless-4.13.0.92
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


Téléchargement automatique des images

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="TA_CLE_PRIVEE_ICI")
project = rf.workspace("ton-nom").project("ton-projet")
dataset = project.version(2).download("yolov8")

# On affiche le chemin pour être sûr
print("Tes images sont ici :", dataset.location)

Entraînement sécurisé sur Drive

In [ ]:
from ultralytics import YOLO

# On charge ton fichier .pt que tu viens d'uploader
model = YOLO('/content/yolov8s (1).pt')

# On lance la machine
model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,
    imgsz=640,
    project='/content/drive/MyDrive/Mangrove_Memoire',
    name='resultats_v1'
)

Générer et Afficher les Graphes

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Chemin vers tes résultats sur le Drive
path = '/content/drive/MyDrive/Mangrove_Memoire/session_1/'

# 1. Courbes d'apprentissage (Loss, mAP)
plt.figure(figsize=(10,10))
plt.imshow(mpimg.imread(path + 'results.png'))
plt.axis('off')
plt.title("Courbes d'apprentissage")
plt.show()

# 2. Matrice de Confusion (Pour voir les erreurs)
plt.figure(figsize=(10,10))
plt.imshow(mpimg.imread(path + 'confusion_matrix.png'))
plt.axis('off')
plt.title("Matrice de Confusion")
plt.show()

Le Test Final

In [ ]:
# Charger le MEILLEUR modèle sauvegardé
best_model = YOLO('/content/drive/MyDrive/Mangrove_Memoire/session_1/weights/best.pt')

# Lancer le test sur les images de test
metrics = best_model.val(split='test')

print(f"Précision sur le dossier Test (mAP50): {metrics.box.map50}")

Visualiser des prédictions réelles

In [ ]:
# Faire une prédiction sur les images de test
results = best_model.predict(source=f"{dataset.location}/test/images", save=True, conf=0.5)

# Afficher une des images prédites
import glob
prediction_images = glob.glob('/content/drive/MyDrive/Mangrove_Memoire/session_1/predict/*.jpg')
if prediction_images:
    plt.imshow(mpimg.imread(prediction_images[0]))
    plt.axis('off')
    plt.show()